# **Explaining a linear regression model**
Before using Shapley values to explain complicated models, it is helpful to understand how they work for simple models. One of the simplest model types is standard linear regression, and so below we train a linear regression model on the classic [boston housing dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). This dataset consists of 506 neighboorhood regions around Boston in 1978, where our goal is to predict the median home price (in thousands) in each neighboorhood from 14 different features:

1. CRIM - per capita crime rate by town
2. ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS - proportion of non-retail business acres per town.
4. CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
5. NOX - nitric oxides concentration (parts per 10 million)
6. RM - average number of rooms per dwelling
7. AGE - proportion of owner-occupied units built prior to 1940
8. DIS - weighted distances to five Boston employment centres
9. RAD - index of accessibility to radial highways
10. TAX - full-value property-tax rate per \$10,000
11. PTRATIO - pupil-teacher ratio by town
12. B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. LSTAT - % lower status of the population
14. MEDV - Median value of owner-occupied homes in $1000’s

In [1]:
%%capture
!pip install shap==0.39

In [2]:
import pandas as pd
import shap
import sklearn
from ipywidgets import interact
from sklearn.model_selection import train_test_split

# a classic housing price dataset
X,y = shap.datasets.boston()
X_train, X_test, Y_train, Y_test = train_test_split(*shap.datasets.boston(), test_size=0.1, random_state=0)
# X100 = shap.utils.sample(X, 100) # 100 instances for use as the background distribution

# a simple linear model
model = sklearn.linear_model.LinearRegression()
model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Examining the model coefficients
The most common way of understanding a linear model is to examine the coefficients learned for each feature. These coefficients tell us how much the model output changes when we change each of the input features:

In [3]:
print("Model coefficients:\n")
for i in range(X_train.shape[1]):
    print(X_train.columns[i], "=", model.coef_[i].round(4))

Model coefficients:

CRIM = -0.1146
ZN = 0.0362
INDUS = 0.0065
CHAS = 2.1992
NOX = -15.911
RM = 4.268
AGE = -0.0102
DIS = -1.347
RAD = 0.2712
TAX = -0.0116
PTRATIO = -1.0171
B = 0.0098
LSTAT = -0.4438


While coefficients are great for telling us what will happen when we change the value of an input feature, by themselves, they are not a great way to measure the overall importance of a feature. This is because the value of each coefficient depends on the scale of the input features. If for example we were to measure the age of a home in minutes instead of years, then the coefficients for the AGE feature would become 0.0007∗365∗24∗60=367.92. Clearly the number of minutes since a house was built is not more important than the number of years, yet its coefficient value is much larger. This means that the magnitude of a coefficient is not necessarily a good measure of a feature’s importance in a linear model.

## **A more complete picture using partial dependence plots**

To understand a feature’s importance in a model it is necessary to understand both how changing that feature impacts the model’s output, and also the distribution of that feature’s values. To visualize this for a linear model we can build a classical partial dependence plot and show the distribution of feature values as a histogram on the x-axis:

In [4]:
@interact(feature=X_train.columns)
def partial_dependence(feature="RM"):
  shap.plots.partial_dependence(
      feature, model.predict, X_test, ice=False,
      model_expected_value=True, feature_expected_value=True
  )

interactive(children=(Dropdown(description='feature', index=5, options=('CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', …

The gray horizontal line in the plot above represents the expected value of the model when applied to the boston housing dataset. The vertical gray line represents the average value of the AGE feature. Note that the blue partial dependence plot line (which the is average value of the model output when we fix the AGE feature to a given value) always passes through the interesection of the two gray expected value lines. We can consider this intersection point as the “center” of the partial dependence plot with respect to the data distribution. The impact of this centering will become clear when we turn to Shapley values next.

## Reading SHAP values from partial dependence plots
The core idea behind Shapley value based explanations of machine learning models is to use fair allocation results from cooperative game theory to allocate credit for a model’s output $f(x)$ among its input features. In order to connect game theory with machine learning models it is nessecary to both match a model’s input features with players in a game, and also match the model function with the rules of the game. Since in game theory a player can join or not join a game, we need a way for a feature to “join” or “not join” a model. The most common way to define what it means for a feature to “join” a model is to say that feature has “joined a model” when we know the value of that feature, and it has not joined a model when we don’t know the value of that feature. To evaluate an existing model $f$ when only a subset $S$ of features are part of the model we integrate out the other features using a conditional expectated value formulation. This formulation can take two forms:

$$E[f(X)|X_S=x_s]$$
$$or$$
$$E[f(X)|do(X_S=x_s)]$$

In the first form we know the values of the features in S because we observe them. In the second form we know the values of the features in S because we set them. In general, the second form is usually preferable, both becuase it tells us how the model would behave if we were to intervene and change its inputs, and also because it is much easier to compute. In this tutorial we will focus entirely on the the second formulation. We will also use the more specific term SHAP values to refer to Shapley values applied to a conditional expectation function of a machine learning model.

SHAP values can be very complicated to compute (they are NP-hard in general), but linear models are so simple that we can read the SHAP values right off a partial dependence plot. When we are explaining a prediction 𝑓(𝑥), the SHAP value for a specific feature 𝑖 is just the difference between the expected model output and the partial dependence plot at the feature’s value 𝑥𝑖:

In [5]:
# compute the SHAP values for the linear model
explainer = shap.Explainer(model.predict, X_test)
shap_values = explainer(X_test)

In [6]:
# make a standard partial dependence plot
@interact(feature=X_test.columns, sample_ind=list(range(X_test.shape[0])))
def partial_dependance_plot(feature="RM", sample_ind=18):
  shap.partial_dependence_plot(
      feature, model.predict, X_test, model_expected_value=True,
      feature_expected_value=True, ice=False,
      shap_values=shap_values[sample_ind:sample_ind+1,:]
  )

interactive(children=(Dropdown(description='feature', index=5, options=('CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', …

The close correspondence between the classic partial dependence plot and SHAP values means that if we plot the SHAP value for a specific feature across a whole dataset we will exactly trace out a mean centered version of the partial dependence plot for that feature:

In [7]:
@interact(feature=X_train.columns)
def scatter(feature="RM"):
  shap.plots.scatter(shap_values[:,feature])

interactive(children=(Dropdown(description='feature', index=5, options=('CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', …

## The additive nature of Shapley values
One the fundemental properties of Shapley values is that they always sum up to the difference between the game outcome when all players are present and the game outcome when no players are present. For machine learning models this means that SHAP values of all the input features will always sum up to the difference between baseline (expected) model output and the current model output for the prediction being explained. The easiest way to see this is through a waterfall plot that starts our background prior expectation for a home price $𝐸[𝑓(𝑋)]$, and then adds features one at a time until we reach the current model output $𝑓(𝑥)$:

In [8]:
# the waterfall_plot shows how we get from shap_values.base_values to model.predict(X)[sample_ind]
@interact(sample_ind=list(range(X_test.shape[0])))
def waterfall(sample_ind=18):
  shap.plots.waterfall(shap_values[sample_ind], max_display=14)

interactive(children=(Dropdown(description='sample_ind', index=18, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, …

# **END**